In [1]:
import numpy as np
import os 
from os import walk

def load_safari(folder):

    mypath = os.path.join("./data", folder)
    txt_name_list = []
    for (dirpath, dirnames, filenames) in walk(mypath):
        for f in filenames:
            if f != '.DS_Store':
                txt_name_list.append(f)
                break

    slice_train = int(80000/len(txt_name_list))  ###Setting value to be 80000 for the final dataset
    i = 0
    seed = np.random.randint(1, 10e6)

    for txt_name in txt_name_list:
        txt_path = os.path.join(mypath,txt_name)
        x = np.load(txt_path)
        x = (x.astype('float32') - 127.5) / 127.5
        # x = x.astype('float32') / 255.0
        
        x = x.reshape(x.shape[0], 28, 28, 1)
        
        y = [i] * len(x)  
        np.random.seed(seed)
        np.random.shuffle(x)
        np.random.seed(seed)
        np.random.shuffle(y)
        x = x[:slice_train]
        y = y[:slice_train]
        if i != 0: 
            xtotal = np.concatenate((x,xtotal), axis=0)
            ytotal = np.concatenate((y,ytotal), axis=0)
        else:
            xtotal = x
            ytotal = y
        i += 1
        
    return xtotal, ytotal

In [2]:
import torch
import torch.nn as nn

class View(nn.Module):
    def __init__(self,shape):
        super().__init__()
        self.shape = shape,
    def forward(self,x):
        return x.view(*self.shape)

In [28]:
from torch.autograd import grad,Variable

class D(nn.Module):
    def __init__(self,dim_latent=200):
        # 파이토치 부모 클래스 초기화
        super().__init__()
        self.dim_latent = dim_latent
       # 신경망 레이어 정의
        self.model = nn.Sequential(
            #(-1,1,28,28)
            nn.Conv2d(1, 64, kernel_size=7,stride=2,padding=3),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            #(-1,64,14,14)
            
            nn.Conv2d(64, 64, kernel_size=5, stride=2,padding=2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            #(-1,64,7,7)

            nn.Conv2d(64, 128, kernel_size=4),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            #(-1,128,4,4)
            
            nn.Conv2d(128, 128, kernel_size=3,padding=1),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            #(-1,128,4,4)

            View((-1,128*4*4)),
            nn.Linear(128*4*4, 1),
        )


        # 손실 함수 생성
        
        # 옵티마이저 생성
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0005)
    def forward(self,x):
        return self.model(x)

    def wasserstein_loss(self,y_pred,y_true):
        return torch.mean(y_true*y_pred)

    def gradient_penalty_loss(self,x_batch,fake_img):
        batch_size = x_batch.shape[0]

        # Calculate interpolation
        alpha = torch.rand(batch_size, 1, 1, 1)
        alpha = alpha.expand_as(x_batch)
        
        #alpha = alpha.cuda()
        interpolated = alpha * x_batch.data + (1 - alpha) * fake_img.data
        interpolated = Variable(interpolated, requires_grad=True)        
        #interpolated=interpolated.cuda()
        y_pred = self.forward(interpolated)

        gradients = grad(outputs=y_pred, inputs=interpolated,
                               grad_outputs=torch.ones(y_pred.size()).cuda() if torch.cuda.is_available() else torch.ones(
                               y_pred.size()),
                               create_graph=True, retain_graph=True)[0] #[64,1,28,28]

        gradients = gradients.view(batch_size, -1)
        gradients_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)
        gp_loss = ((gradients_norm - 1) ** 2).mean()

        self.optimiser.zero_grad()
        gp_loss.backward()
        self.optimiser.step()
        return gp_loss

    def train(self,x_batch,label):
        self.optimiser.zero_grad()
        y_pred = self.forward(x_batch)

        

        loss = self.wasserstein_loss(y_pred,label)
        loss.backward()
        self.optimiser.step()
        return loss
        ## weight clipping하면 clipped grad 어떻게 계산되지?

In [24]:
class G(nn.Module):
    def __init__(self,dim_latent=100):
        # 파이토치 부모 클래스 초기화
        super().__init__()
       # 신경망 레이어 정의
        self.dim_latent = dim_latent
        self.model = nn.Sequential(
            #(-1,100)
            nn.Linear(self.dim_latent,64*7*7),
            View((-1,64,7,7)),
            #(-1,3136)
            #(-1,64,7,7)
            
            nn.ConvTranspose2d(64,64,4,stride=2,padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),            
            #(-1,64,14,14)

            nn.ConvTranspose2d(64,128,5,padding=2),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            #(-1,128,14,14)

            nn.ConvTranspose2d(128,64,6,stride=2,padding=2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),
            #(-1,64,28,28)

            nn.ConvTranspose2d(64,1,5,padding=2),
            nn.Tanh(),
            #(-1,1,28,28)

        )


        # 옵티마이저 생성
        self.optimiser = torch.optim.Adam(self.parameters(), lr=0.0001)
    def forward(self,x):
        return self.model(x)

    def train(self,D,latent_batch,label):
    
        self.optimiser.zero_grad()
        ## D.optimiser.zero_grad()  필요없음
        g_output = self.forward(latent_batch)
        d_output = D(g_output)

        loss = D.wasserstein_loss(d_output,label)
        loss.backward()
        self.optimiser.step()
        # D(x_input,1) ->D학습
        # D(G(latent), 0) ->D학습
        # D(G(latent),1) ->G학습
        return loss


In [25]:
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np

class MyDataset(Dataset):
    def __init__(self,foldername='camel'):
        (self.x_train, self.y_train) =load_safari(foldername)
    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        
        x_train = self.x_train.transpose((0,3,1,2))
        return torch.FloatTensor(x_train[idx])
        #return torch.cuda.FloatTensor(self.x_train[idx])
#data_loader = DataLoader(MyDataset(), batch_size=128, shuffle=True,generator=torch.Generator(device='cuda'))
data_loader = DataLoader(MyDataset(), batch_size=64, shuffle=True)


In [29]:


d = D()
g= G()
#d.to(device)
#g.to(device)

d_loss_hist = []
g_loss_hist =[]
for epoch in range(1, 101):
    for i,x_input in enumerate(data_loader):

        pass
        label = torch.ones((x_input.shape[0],1))
        fake_label = -torch.ones((x_input.shape[0],1))
        d_loss=d.train(x_input,label)
        fake_img_batch = g(torch.randn(64,100)).detach()
        d_fake_loss=d.train(fake_img_batch,fake_label)
        gp_loss = d.gradient_penalty_loss(x_input,fake_img_batch)
        # gp_loss(fake_img_batch, x_input)
        ## interpolated_img = interpolate_img(fake_img_batch,x_input)
        ## d.optimizer.zero_grad()
        ## d.forward(interpolated_img)
        ## gradient 다 모으고
        ## L2_norm ==1 되도록 loss
        break
    break
    #     total_d_loss =(d_loss.item()+d_fake_loss.item()+gp_loss.item())/2
    #     d_loss_hist.append(total_d_loss)
    #     if (i%5==4):
    #         g_loss=g.train(d,torch.randn(64,100),label)
    #         g_loss_hist.append(g_loss.item())
    #         print(f"Epoch {epoch}: d_loss = {total_d_loss:.9f}  g loss={g_loss.item():.9f} gp loss={gp_loss.item():.9f}")
    # if epoch % 1 == 0:
    #     print(f"Epoch {epoch}: d_loss = {total_d_loss:.9f}  g loss={g_loss.item():.9f}")


tensor([[[-0.1174, -0.0538, -0.1155, -0.0917,  0.0879, -0.0789,  0.0230],
         [ 0.0156, -0.0600, -0.0615,  0.0473, -0.0534,  0.0918, -0.0627],
         [-0.0437,  0.0755, -0.0291, -0.1245,  0.1260, -0.1191,  0.0248],
         [-0.0515,  0.0629,  0.1147, -0.0463,  0.0073, -0.0604, -0.0442],
         [-0.0516,  0.1370,  0.0958,  0.0421,  0.0810, -0.1087, -0.0011],
         [-0.0214, -0.0704,  0.1094, -0.1189, -0.1049, -0.0863,  0.0132],
         [-0.0530, -0.1027,  0.0043,  0.0497, -0.0191, -0.1281, -0.0105]]],
       requires_grad=True)
<class 'tuple'> 1
torch.Size([64, 1, 28, 28])


IndexError: tuple index out of range

In [ ]:
import matplotlib.pyplot as plt

fake_pred = g.forward(torch.randn(8,100)).detach().cpu().numpy() #(8,1,28,28)
fake_pred = fake_pred.transpose(0,2,3,1) #(8,28,28,1)
fake_8img = fake_pred.transpose(1,0,2,3).reshape(28,-1,1)
plt.figure(figsize=(16,2))
plt.imshow(fake_8img)

In [ ]:
def visualize(loss_hist):
    plt.figure(figsize=(12,4))
    plt.scatter(np.arange(1,len(loss_hist)+1),loss_hist,s=0.5)
    plt.title('generator loss')
    plt.show()
visualize(g_loss_hist)
visualize(d_loss_hist)